## Analísis de los Datasets de densidad 

In [1]:
import pandas as pd

In [2]:
density_folder= '../Files/density/'

In [3]:
df_densidad = (pd.read_csv(f'{density_folder}01-2023.csv', sep = ";"))
df_densidad['vmed'] = df_densidad['vmed'].fillna(0)

Total Rows:

In [4]:
df_densidad.shape[0]

12644350

vmed nulls

In [5]:
df_densidad[df_densidad['vmed'].isnull()].shape[0]

0

intensidad nulls

In [6]:
df_densidad[df_densidad['intensidad'].isnull()].shape[0]

0

Check carga nulls

In [7]:
df_densidad[df_densidad['carga'].isnull()].shape[0]

0

In [8]:
df_densidad[~(df_densidad['error'].isnull() | (df_densidad['error'] == 'N'))].shape[0]

0

Total vmed 0s

In [9]:
df_densidad[df_densidad['vmed']== 0 ].shape[0]

11498052

Calcular porcentaje de 0s

In [10]:
pct_vmed_zero = (df_densidad[df_densidad['vmed']== 0 ].shape[0] /  df_densidad.shape[0])  * 100  # Calculate the percentage
print(f"The percentage of rows where 'vmed' is 0 is: {pct_vmed_zero:.2f}%")

The percentage of rows where 'vmed' is 0 is: 90.93%


In [11]:
def merge_dataset(month,density_folder):
    #This program will join the density data which its density location file for each month

    print(f'start processing month {month}')
    #Open the datasets
    df_ubicacion = pd.read_csv(f'{density_folder}pmed_ubicacion_{month}-2023.csv', sep= ";" )
    df_densidad = pd.read_csv(f'{density_folder}{month}-2023.csv', sep = ";")
    
    #df_densidad['vmed'] = df_densidad['vmed'].fillna(0)
    df_densidad[['fecha', 'hora']] = df_densidad['fecha'].str.split(' ', expand=True)
    df_densidad['int_real'] = df_densidad['intensidad']/4 #As per documentation "intensidad" is marked as vehicles/hour so to get the real intensidad in 15 min intervals we need to divide by four
    df_ubicacion['cod_distrito'] = df_ubicacion['distrito']  
    
    print(f'before merge month {month}')
    
    df = (
        pd.merge(df_densidad, df_ubicacion, on='id')[['fecha' ,  'cod_distrito'  , 'int_real' , 'ocupacion' , 'vmed' , 'hora']]
        .groupby(['cod_distrito', 'fecha' , 'hora' ]).agg({
                                    'int_real': 'mean',
                                    'ocupacion': 'mean',
                                    'vmed':  lambda x: x[x > 0].mean()})
        .assign(vmed=lambda x: x['vmed'].fillna(0))                                    
        .reset_index()
    )       
    
    print(f'finish processing month {month}')
    return df

In [12]:
dfs= []
for i in range(1, 6):
    df = merge_dataset(f"{i:02}" , density_folder)
    dfs.append(df)
    
if len(dfs)>0:
    dfd = pd.concat(dfs , ignore_index= True)    

dfdg =dfd.groupby(['cod_distrito', 'fecha' , 'hora' ]).agg({
                                        'int_real': 'mean',
                                        'ocupacion': 'mean',
                                        'vmed':  lambda x: x[x > 0].mean()}).assign(vmed=lambda x: x['vmed'].fillna(0)).reset_index()

#dfdg.to_csv('../Files/2023_densidad.csv' , sep = ";" , index=False )    

start processing month 01
before merge month 01
finish processing month 01
start processing month 02
before merge month 02
finish processing month 02
start processing month 03
before merge month 03
finish processing month 03
start processing month 04
before merge month 04
finish processing month 04
start processing month 05
before merge month 05
finish processing month 05


Vemos las mediciones por distrito. Se puede ver que hay muchos puntos de medición que dan 0, pero algunos distritos si muestran una métrica correcta, entre 50-75 kmh

In [13]:
dfdg.groupby(['cod_distrito']).agg({'vmed':lambda x: x[x > 0].mean()}).assign(vmed=lambda x: x['vmed'].fillna(0))

,vmed
cod_distrito,
1.0,56.981803
2.0,60.312310
3.0,69.157575
4.0,73.263078
5.0,68.553823
6.0,0.000000
7.0,0.000000
8.0,66.716198
9.0,66.752491


In [22]:
pct_vmed_zero = (dfd[dfd['vmed']== 0 ].shape[0] /  df_densidad.shape[0])  * 100  # Calculate the percentage
print(f"The percentage of rows where 'vmed' is 0 is: {pct_vmed_zero:.2f}%")

The percentage of rows where 'vmed' is 0 is: 0.81%


In [29]:
dfdg.shape[0]

21

In [30]:
dfdg[dfdg['vmed'].isnull()].shape[0]

0

In [31]:
pct_vmed_zero = (df_densidad[df_densidad['vmed']== 0 ].shape[0] /  df_densidad.shape[0])  * 100  # Calculate the percentage
print(f"The percentage of rows where 'vmed' is 0 is: {pct_vmed_zero:.2f}%")

The percentage of rows where 'vmed' is 0 is: 90.93%
